In [1]:
import os
from datetime import datetime
# Make logging folder named according to current time
start_time_str = datetime.today().strftime('%d_%m_%Y__%H_%M_%S')
log_dir = './logs/'f"log_{start_time_str}"
os.makedirs(log_dir)

# Initialize logger that adds to text file in current logging folder
from logging_utils import *

init_logger(save_dir=log_dir)
logger = get_logger()




In [2]:
import pathlib
import subprocess
# Get the absolute path to your repository, 
# no matter where you are running this code from

repo_path = os.getcwd() 


git_branch = subprocess.check_output(
    ["git", "-C", repo_path, "rev-parse", "--abbrev-ref", "HEAD"]).strip().decode('UTF-8')

git_commit_short_hash = subprocess.check_output(
    ["git", "-C", repo_path, "describe", "--always"]).strip().decode('UTF-8')


In [3]:
import numpy as np

dim=200
x0=np.squeeze(np.zeros((dim,1)))
x0Type='Zero Vector'

FD_StencilMat=np.zeros((dim,dim))
b = np.ones(dim-1)
c = -2*np.ones(dim)
np.fill_diagonal(FD_StencilMat[1:], b)
np.fill_diagonal(FD_StencilMat[:,1:], b)
np.fill_diagonal(FD_StencilMat, c)
A=FD_StencilMat
AType='1D Laplacian'


nmax_iter = dim
retrain_freq=20
e1 = 1e-8
e2 = 1e-10
nmax_iter = 50
restart   = 64

n_steps =100
InputDim=dim
HiddenDim=2*dim
HiddenDim2=2*dim
OutputDim=dim

In [4]:
# Initial log message
logger.info("Add git hash to logging and clean up sanbox before benchmarking")

2020-05-12 12:55:49 [INFO] <ipython-input-4-21a5bd66ee3b> 2 - Add git hash to logging and clean up sanbox before benchmarking


In [5]:
# Set up parameter class
class MyParamsClass():
    def __init__(self, AType,dim,nmax_iter,restart,retrain_freq,e1,e2,x0Type,n_steps,InputDim,HiddenDim,HiddenDim2,OutputDim,git_branch,git_commit_short_hash):
        self.AType = AType
        self.dim = dim
        self.nmax_iter = nmax_iter
        self.restart = restart
        self.n_steps =n_steps
        self.retrain_freq=retrain_freq
        self.e1=e1
        self.e2=e2
        self.x0Type=x0Type
        self.InputDim=InputDim
        self.HiddenDim=HiddenDim
        self.HiddenDim2=HiddenDim2
        self.OutputDim=OutputDim
        self.git_branch=git_branch
        self.git_commit_short_hash=git_commit_short_hash

In [6]:
import os
import json

params = MyParamsClass(AType,dim,nmax_iter,restart,retrain_freq,e1,e2,x0Type,n_steps,InputDim,HiddenDim,HiddenDim2,OutputDim,git_branch,git_commit_short_hash)
sorted_params_dict = {k: params.__dict__[k] for k 
                      in sorted(params.__dict__.keys())}

params_filepath = log_dir+'/params.json'
json.dump(sorted_params_dict, open(params_filepath, 'w'), indent=4)

In [7]:
from src_dir import *
import time

In [8]:
@nn_preconditioner(retrain_freq=retrain_freq, debug=True,InputDim=InputDim,HiddenDim=HiddenDim,HiddenDim2=HiddenDim2,OutputDim=OutputDim)
def MLGMRES(A, b, x0, e, nmax_iter,IterErr0_sum,IterErr0,ProbCount,Add,restart,debug):
    return GMRES(A, b, x0, e, nmax_iter,restart, debug)
    

In [9]:
IterErr0_sum=0.0 
IterErr0_sum2=0.0
IterErr0=0
debug=True
Add1=False
Add2=True

for ProbIdx in range(n_steps):
    b =6*np.squeeze(np.random.randn(1,dim))  #random b
    b_norm=np.linalg.norm(b)
    Out,IterErr0_sum,IterErr0=MLGMRES(A, b/b_norm, x0, e1, nmax_iter,IterErr0_sum,IterErr0,ProbIdx,Add1,restart,debug)  # normalize b for optimal NN performance.
    Out2,IterErr0_sum,IterErr0=MLGMRES(A, b/b_norm, Out[-1], e2, nmax_iter,IterErr0_sum,IterErr0,ProbIdx,Add2,restart,debug)
    print(ProbIdx)





0.6067414695964105 inf
0
0.5912960815726542 1.1980375511690649
1
0.6367994679093814 0.9174185095392231
2
0.6705904971728447 0.8351425054170969
3
0.6240535504985946 0.7823702666874713
4
0.6331628864469118 0.7525287906393594
5
0.557879893436582 0.7200873077722298
6
0.663870436839925 0.7120563262104721
7
0.6586576427492298 0.7053814907778168
8
0.6036647397305294 0.6940796295503404
9
0.6144839595833811 0.6861200625536444
10
0.6780716563496062 0.6853883892623682
11
0.6011773681313203 0.6783708041681141
12


KeyboardInterrupt: 

In [ ]:

for ProbIdx in range(n_steps):
    b =6*np.squeeze(np.random.randn(1,dim))  #random b
    b_norm=np.linalg.norm(b)
    NonML_Out1=GMRES(A, b/b_norm, x0, e1, nmax_iter,restart, debug)  # normalize b for optimal NN performance.
    NonML_Out2=GMRES(A, b/b_norm, Out[-1], e2, nmax_iter,restart, debug)  # normalize b for optimal NN performance.
    print(ProbIdx)